In [ ]:
import os
import re
import string
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np

# Comparison between different models
1. Baseline with GPT 3.5 finetuned (ft:gpt-3.5-turbo-1106:ninjatech-ai-dev::8sQyzUKb)
2. Baseline with GPT 4
3. Prompt2Code with GPT3.5
4. Prompt2Code with GPT4

In [ ]:

result_baseline_with_GPT3_5_finetuned = [
    {'success': 8, 'wrong_answer': 38, 'error': 0}
]

result_baseline_with_GPT4 = [
    {'success': 8, 'wrong_answer': 38, 'error': 0}
]

result_prompt_GPT3_5_reflexion = [
    {'success': 30, 'wrong_answer': 16, 'error': 0},
    {'success': 28, 'wrong_answer': 18, 'error': 0},
    {'success': 33, 'wrong_answer': 13, 'error': 0},
    {'success': 33, 'wrong_answer': 13, 'error': 0},
    {'success': 27, 'wrong_answer': 19, 'error': 0}
]


result_prompt_GPT4 = [
    {'success': 32, 'wrong_answer': 12, 'error': 2}
]


# Calculate average success rate and average error rate for each list
def calculate_average_rates(result_list):
    total_records = len(result_list)
    if total_records == 0:
        return 0, 0
    total_questions = sum(result_list[0].values())
    success_sum = sum(result['success'] for result in result_list)
    error_sum = sum(result['error'] for result in result_list)
    average_success_rate = success_sum / total_records/total_questions
    average_exec_rate = 1- error_sum / total_records/total_questions
    return average_success_rate, average_exec_rate

success_baseline_GPT3_5, execution_baseline_GPT3_5 = calculate_average_rates(result_baseline_with_GPT3_5_finetuned)
success_baseline_with_GPT4, execution_baseline_with_GPT4 = calculate_average_rates(result_baseline_with_GPT4)
success_code_GPT3_5_reflexion, execution_code_GPT3_5_reflexion = calculate_average_rates(result_prompt_GPT3_5_reflexion)
success_code_GPT4, execution_code_GPT4 = calculate_average_rates(result_prompt_GPT4)


fig, ax = plt.subplots()

fruits = ['Baseline_GPT3_5_finetuned', 'Baseline_GPT4', 'code_GPT3_5_reflexion', 'code_GPT4']
counts = [40, 100, 30, 55]
bar_labels = ['red', 'blue', '_red', 'orange']
bar_colors = ['tab:red', 'tab:blue', 'tab:red', 'tab:orange']

ax.bar(fruits, counts, label=bar_labels, color=bar_colors)

ax.set_ylabel('fruit supply')
ax.set_title('Fruit supply by kind and color')
ax.legend(title='Fruit color')

plt.show()